In [2]:
from __future__ import print_function
import json
from pymongo import MongoClient
import numpy as np
import pandas as pd
from datetime import datetime
import time
from datetime import timedelta
from bs4 import BeautifulSoup as bs
import requests
import csv
import nltk
import seaborn as sns
import matplotlib.pyplot as plt

# Carico 2 csv per generare df statiUS_capoluogo e statiCA_capoluogo

In [3]:
legendaUS=pd.read_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\citiesUS.csv",
                    engine="python", encoding="utf-8")
legendaCA=pd.read_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\citiesCA.csv",
                    engine="python", encoding="utf-8")

In [29]:
legendaUS.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314
2,Packwood,Packwood,WA,Washington,53041,Lewis,46.6085,-121.6702,NaN,NaN,213.9,polygon,False,America/Los_Angeles,98361,1840025265
3,Wautauga Beach,Wautauga Beach,WA,Washington,53035,Kitsap,47.5862,-122.5482,NaN,NaN,261.7,point,False,America/Los_Angeles,98366,1840037725
4,Harper,Harper,WA,Washington,53035,Kitsap,47.5207,-122.5196,NaN,NaN,342.1,point,False,America/Los_Angeles,98366,1840037659


In [30]:
legendaCA.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Toronto,43.666667,-79.416667,Canada,CA,Ontario,admin,5213000,3934421
1,Montréal,45.500000,-73.583333,Canada,CA,Québec,NaN,3678000,2356556
2,Vancouver,49.250000,-123.133333,Canada,CA,British Columbia,NaN,2313328,603502
3,Ottawa,45.416667,-75.700000,Canada,CA,Ontario,primary,1145000,812129
4,Calgary,51.083333,-114.083333,Canada,CA,Alberta,NaN,1110000,915322


In [31]:
legendaUS=legendaUS[["city","state_id","state_name","lat","lng","population"]]
legendaUS.head()

iniziali=legendaUS["state_id"].unique()
stato=legendaUS["state_name"].unique()

statiUs=pd.DataFrame(columns=["id","state","country"],data={'id':iniziali,'state':stato,"country":"USA"})
statiUs=statiUs.sort_values(by=["state"])
legendaUS["city"]=legendaUS["city"].apply(lambda stringa: stringa.title())
statiUs.head()

,id,state,country
26,AL,Alabama,USA
25,AK,Alaska,USA
34,AZ,Arizona,USA
27,AR,Arkansas,USA
20,CA,California,USA


In [32]:
legendaUS["population"]=legendaUS["population"].fillna(0)

In [33]:
legendaCA=legendaCA[["city","iso2","admin","population","lat","lng"]]
legendaCA.head()
statiCA_capoluogo=legendaCA.loc[legendaCA.groupby("admin")["population"].idxmax()]

In [34]:
%store legendaCA

Stored 'legendaCA' (DataFrame)


In [35]:
stato=legendaCA["admin"].unique()
statiCA=pd.DataFrame(columns=["state","country"],data={'state':stato,"country":"CAN"})
legendaCA=legendaCA.drop(columns="iso2")
statiCA.head()

,state,country
0,Ontario,CAN
1,Québec,CAN
2,British Columbia,CAN
3,Alberta,CAN
4,Manitoba,CAN


In [36]:
statiCA_capoluogo

,city,iso2,admin,population,lat,lng
4,Calgary,CA,Alberta,1110000,51.083333,-114.083333
2,Vancouver,CA,British Columbia,2313328,49.250000,-123.133333
7,Winnipeg,CA,Manitoba,632063,49.883333,-97.166667
27,Moncton,CA,New Brunswick,90635,46.096520,-64.797570
22,Saint John’s,CA,Newfoundland and Labrador,131469,47.550000,-52.666667
72,Yellowknife,CA,Northwest Territories,19234,62.450000,-114.350000
11,Halifax,CA,Nova Scotia,359111,44.650000,-63.600000
114,Iqaluit,CA,Nunavut,6124,63.733333,-68.500000
0,Toronto,CA,Ontario,5213000,43.666667,-79.416667
46,Charlottetown,CA,Prince Edward Island,42402,46.238225,-63.139481


In [37]:
statiCA_capoluogo.columns=["city","iso2","state","pop","lat","lng"]

statiCA_capoluogo=pd.merge(statiCA_capoluogo,statiCA,on="state",how="inner")

In [38]:
statiCA_capoluogo=statiCA_capoluogo.drop(columns=["iso2","pop"])
statiCA_capoluogo["iso3"]="CAN"

In [39]:
statiUS_capoluogo=legendaUS.loc[legendaUS.groupby("state_name")["population"].idxmax()]
statiUS_capoluogo["iso3"]="USA"

# Memorizzo i due df creati precedentemente che verranno utilizzati nel notebook sistemazione_pos

In [40]:
% store statiUS_capoluogo

Stored 'statiUS_capoluogo' (DataFrame)


In [41]:
%store statiCA_capoluogo

Stored 'statiCA_capoluogo' (DataFrame)


In [14]:
statiCA_capoluogo.head()

,city,state,lat,lng,country
0,Calgary,Alberta,51.083333,-114.083333,CAN
1,Vancouver,British Columbia,49.250000,-123.133333,CAN
2,Winnipeg,Manitoba,49.883333,-97.166667,CAN
3,Moncton,New Brunswick,46.096520,-64.797570,CAN
4,Saint John’s,Newfoundland and Labrador,47.550000,-52.666667,CAN
